In [1]:
import re
import numpy as np
import pandas as pd
from ipywidgets import interact

In [2]:
## obtain user data and review data
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import MySQLdb
import pandas as pd
sshtunnel.SSH_TIMEOUT = 50.0
sshtunnel.TUNNEL_TIMEOUT = 50.0

with SSHTunnelForwarder(
         'grande.wwbp.org',
         ssh_pkey='/Users/rahuldhakecha/.ssh/id_rsa',
         ssh_username="rahuldhakecha",
         remote_bind_address=('127.0.0.1', 3306)) as server:
    con = None
    con = MySQLdb.connect(user='rahuldhakecha',passwd='wCPdeAk:zcpQvw8p',db='goodreads',host='127.0.0.1',port=server.local_bind_port)
    query = 'select * from users_2000_review_details;'
    review_data = pd.read_sql_query(query, con)
    con.commit()
    query = 'select * from users_2000_book_details;'
    user_data = pd.read_sql_query(query, con)
    con.commit()
    con.close()

In [3]:
review_data_temp=review_data
user_data_temp=user_data

In [5]:
review_data.rename(columns={'review': 'review_text'}, inplace=True)

In [6]:
# review_data[review_data['book_id']=='16219975"><img alt="']
def replace_string(x):
    x=int(re.findall(r'\d+', x)[0])
    return(x)
review_data['book_id'] = review_data['book_id'].apply(replace_string)

In [7]:
review_data['user_id'] = review_data['user_id'].astype(int, errors='ignore')
review_data['book_id'] = review_data['book_id'].astype(int, errors='ignore')
user_data['user_id'] = user_data['user_id'].astype(int, errors='ignore')
user_data['book_id'] = user_data['book_id'].astype(int, errors='ignore')

In [8]:
# user_review_data = user_data.join(review_data, on=['user_id','book_id'], how='inner')
user_review_data = pd.merge(user_data, review_data, on=['user_id','book_id'])

In [9]:
user_review_data

,user_id,book_id,user_rating,avg_rating,book_title,author,rating,review,genre,review_id,review_text
0,2636606,54319,3,3.64,Being Teddy Roosevelt,Claudia Mills,200,34,Childrens; Childrens; Juvenile; Realistic Fict...,2181897791,Mills' sweet spot is definitely about 4th grad...
1,2636606,7640262,3,3.90,Frankly; Frannie (Frankly; Frannie #)1),A.J. Stern;Doreen Mulryan Marts,516,82,Humor; Fiction; Childrens; Juvenile; Childrens...,2181897375,My quest for early chapter books continues. I ...
2,2636606,34669556,3,3.98,Have Sword; Will Travel,Garth Nix,75,23,Fantasy; Childrens; Middle Grade; Adventure; C...,2178934117,Typical medievalish fantasy with dragons and t...
3,2636606,31193426,3,3.90,The Witch Boy (The Witch Boy #)1),Molly Ostertag;The Witch Boy,808,118,Sequential Art; Graphic Novels; Fantasy; Child...,2178933041,"Nicely done, definitely diverse, lots of actio..."
4,2636606,25741324,3,3.80,Finding Mighty,Sheela Chari,78,30,Childrens; Middle Grade; Mystery; Thriller; My...,2177890961,"Interesting book, lots of diverse characters, ..."
5,2636606,33985145,3,3.76,The Date to Save,Stephanie Kate Strohm,59,20,Young Adult; Contemporary; Humor; Fiction\r,2177862199,Odd format and definitely a high school title....
6,2636606,27414421,3,3.56,Sticker Girl,Janet Tashjian,50,22,Fantasy; Childrens; Middle Grade; Childrens; K...,2177861646,Library copy. Perfect for my struggling 6th gr...
7,2636606,25855604,3,4.20,Out of Tune,Gail Nall,30,7,Childrens; Middle Grade\r,2168911650,"Normally love this author's work, but since I ..."
8,2636606,32202503,3,4.48,From Ant to Eagle,Alex Lyttle,149,60,Realistic Fiction; Contemporary; Childrens; Mi...,2168899027,Definitely on the sad side. I agree with the a...
9,2636606,32739712,3,3.91,Big Machines: The Story of Virginia Lee Burton,Sherri Duskey Rinker,134,46,Childrens; Picture Books; Biography; Nonfictio...,2162034739,"The illustrations were charming, but there rea..."


#### fetch genre of each book

In [10]:
user_review_data=user_review_data[-user_review_data['genre'].isnull()]
## split different genres into different columsn
def get_genre(x, n):
    try:
        return x.split(';')[n-1].lstrip()
    except (AttributeError,TypeError,IndexError):
        return x.split(';')[0].lstrip()
user_review_data['genre1']=user_review_data['genre'].apply(get_genre,args=[1])
user_review_data['genre2']=user_review_data['genre'].apply(get_genre,args=[2])
user_review_data['genre3']=user_review_data['genre'].apply(get_genre,args=[3])
user_review_data['genre4']=user_review_data['genre'].apply(get_genre,args=[4])
user_review_data['genre5']=user_review_data['genre'].apply(get_genre,args=[5])
user_review_data['genre6']=user_review_data['genre'].apply(get_genre,args=[6])
# pd.options.display.max_rows = 4000
old_new_genres=pd.read_csv('/Users/rahuldhakecha/coursesfall2017/goodreads/files/top_2000_user_perspective/unique_genres_reduced_set_JE.csv')
old_new_genres=old_new_genres.drop_duplicates()
#replace with new genre categorizatoion
user_review_data=user_review_data.join(old_new_genres.set_index('Old_genre'),on='genre1')
# del user_review_data['genre1']
user_review_data=user_review_data.rename(columns={'New_genre':'New_genre1'})
user_review_data=user_review_data.join(old_new_genres.set_index('Old_genre'),on='genre2')
# del user_review_data['genre2']
user_review_data=user_review_data.rename(columns={'New_genre':'New_genre2'})
user_review_data=user_review_data.join(old_new_genres.set_index('Old_genre'),on='genre3')
# del user_review_data['genre3']
user_review_data=user_review_data.rename(columns={'New_genre':'New_genre3'})
user_review_data=user_review_data.join(old_new_genres.set_index('Old_genre'),on='genre4')
# del user_review_data['genre4']
user_review_data=user_review_data.rename(columns={'New_genre':'New_genre4'})
user_review_data=user_review_data.join(old_new_genres.set_index('Old_genre'),on='genre5')
# del user_review_data['genre5']
user_review_data=user_review_data.rename(columns={'New_genre':'New_genre5'})
user_review_data=user_review_data.join(old_new_genres.set_index('Old_genre'),on='genre6')
# del user_review_data['genre6']
user_review_data=user_review_data.rename(columns={'New_genre':'New_genre6'})

In [11]:
#Fiction doesn't carry much information; therefore we remove it and replace it with next different categorization
def replace_fiction(x):
    try:
        i=0
        while x[i]=='Fiction':
            i+=1
        return x[i]
    except IndexError:
        return x[0]
user_review_data_wo_fiction=user_review_data
user_review_data_wo_fiction['New_genre']=user_review_data[['New_genre1','New_genre2','New_genre3','New_genre4','New_genre5','New_genre6']].apply(replace_fiction,axis=1)

In [12]:
user_review_data_wo_fiction

,user_id,book_id,user_rating,avg_rating,book_title,author,rating,review,genre,review_id,...,genre4,genre5,genre6,New_genre1,New_genre2,New_genre3,New_genre4,New_genre5,New_genre6,New_genre
0,2636606,54319,3,3.64,Being Teddy Roosevelt,Claudia Mills,200,34,Childrens; Childrens; Juvenile; Realistic Fict...,2181897791,...,Realistic Fiction,Fiction\r,Childrens,Childrens,Childrens,NaN,Fiction,NaN,Childrens,Childrens
1,2636606,7640262,3,3.90,Frankly; Frannie (Frankly; Frannie #)1),A.J. Stern;Doreen Mulryan Marts,516,82,Humor; Fiction; Childrens; Juvenile; Childrens...,2181897375,...,Juvenile,Childrens,Chapter Books\r,Nonfiction,Fiction,Childrens,NaN,Childrens,NaN,Nonfiction
2,2636606,34669556,3,3.98,Have Sword; Will Travel,Garth Nix,75,23,Fantasy; Childrens; Middle Grade; Adventure; C...,2178934117,...,Adventure,Childrens,Fiction\r,Fantasy,Childrens,Childrens,Nonfiction,Childrens,NaN,Fantasy
3,2636606,31193426,3,3.90,The Witch Boy (The Witch Boy #)1),Molly Ostertag;The Witch Boy,808,118,Sequential Art; Graphic Novels; Fantasy; Child...,2178933041,...,Childrens,Middle Grade,Sequential Art\r,Graphic Novels,Graphic Novels,Fantasy,Childrens,Childrens,NaN,Graphic Novels
4,2636606,25741324,3,3.80,Finding Mighty,Sheela Chari,78,30,Childrens; Middle Grade; Mystery; Thriller; My...,2177890961,...,Thriller,Mystery Thriller,Mystery\r,Childrens,Childrens,Mystery Crime Detective,Thriller,NaN,NaN,Childrens
5,2636606,33985145,3,3.76,The Date to Save,Stephanie Kate Strohm,59,20,Young Adult; Contemporary; Humor; Fiction\r,2177862199,...,Fiction\r,Young Adult,Young Adult,YA,Fiction,Nonfiction,NaN,YA,YA,YA
6,2636606,27414421,3,3.56,Sticker Girl,Janet Tashjian,50,22,Fantasy; Childrens; Middle Grade; Childrens; K...,2177861646,...,Childrens,Kids\r,Fantasy,Fantasy,Childrens,Childrens,Childrens,NaN,Fantasy,Fantasy
7,2636606,25855604,3,4.20,Out of Tune,Gail Nall,30,7,Childrens; Middle Grade\r,2168911650,...,Childrens,Childrens,Childrens,Childrens,NaN,Childrens,Childrens,Childrens,Childrens,Childrens
8,2636606,32202503,3,4.48,From Ant to Eagle,Alex Lyttle,149,60,Realistic Fiction; Contemporary; Childrens; Mi...,2168899027,...,Middle Grade,Fiction\r,Realistic Fiction,Fiction,Fiction,Childrens,Childrens,NaN,Fiction,Childrens
9,2636606,32739712,3,3.91,Big Machines: The Story of Virginia Lee Burton,Sherri Duskey Rinker,134,46,Childrens; Picture Books; Biography; Nonfictio...,2162034739,...,Nonfiction,Childrens,Art\r,Childrens,Childrens,Nonfiction,Nonfiction,Childrens,NaN,Childrens


In [13]:
data_text = user_review_data_wo_fiction.drop(['genre1','genre2','genre3','genre4','genre5','genre6','New_genre1','New_genre2','New_genre3','New_genre4','New_genre5','New_genre6'], axis=1)


In [14]:
data_text

,user_id,book_id,user_rating,avg_rating,book_title,author,rating,review,genre,review_id,review_text,New_genre
0,2636606,54319,3,3.64,Being Teddy Roosevelt,Claudia Mills,200,34,Childrens; Childrens; Juvenile; Realistic Fict...,2181897791,Mills' sweet spot is definitely about 4th grad...,Childrens
1,2636606,7640262,3,3.90,Frankly; Frannie (Frankly; Frannie #)1),A.J. Stern;Doreen Mulryan Marts,516,82,Humor; Fiction; Childrens; Juvenile; Childrens...,2181897375,My quest for early chapter books continues. I ...,Nonfiction
2,2636606,34669556,3,3.98,Have Sword; Will Travel,Garth Nix,75,23,Fantasy; Childrens; Middle Grade; Adventure; C...,2178934117,Typical medievalish fantasy with dragons and t...,Fantasy
3,2636606,31193426,3,3.90,The Witch Boy (The Witch Boy #)1),Molly Ostertag;The Witch Boy,808,118,Sequential Art; Graphic Novels; Fantasy; Child...,2178933041,"Nicely done, definitely diverse, lots of actio...",Graphic Novels
4,2636606,25741324,3,3.80,Finding Mighty,Sheela Chari,78,30,Childrens; Middle Grade; Mystery; Thriller; My...,2177890961,"Interesting book, lots of diverse characters, ...",Childrens
5,2636606,33985145,3,3.76,The Date to Save,Stephanie Kate Strohm,59,20,Young Adult; Contemporary; Humor; Fiction\r,2177862199,Odd format and definitely a high school title....,YA
6,2636606,27414421,3,3.56,Sticker Girl,Janet Tashjian,50,22,Fantasy; Childrens; Middle Grade; Childrens; K...,2177861646,Library copy. Perfect for my struggling 6th gr...,Fantasy
7,2636606,25855604,3,4.20,Out of Tune,Gail Nall,30,7,Childrens; Middle Grade\r,2168911650,"Normally love this author's work, but since I ...",Childrens
8,2636606,32202503,3,4.48,From Ant to Eagle,Alex Lyttle,149,60,Realistic Fiction; Contemporary; Childrens; Mi...,2168899027,Definitely on the sad side. I agree with the a...,Childrens
9,2636606,32739712,3,3.91,Big Machines: The Story of Virginia Lee Burton,Sherri Duskey Rinker,134,46,Childrens; Picture Books; Biography; Nonfictio...,2162034739,"The illustrations were charming, but there rea...",Childrens


In [24]:
data_text.book_id.values

array([   54319,  7640262, 34669556, ..., 24486599, 18957036, 22668947])

In [15]:
data_text_analysis = data_text[['user_id','book_id','user_rating','review_id','review_text','New_genre']]
data_text_analysis = data_text_analysis[-data_text_analysis['New_genre'].isnull()]

In [69]:
data_text_analysis

,user_id,book_id,user_rating,review_id,review_text,New_genre
0,2636606,54319,3,2181897791,Mills' sweet spot is definitely about 4th grad...,Childrens
1,2636606,7640262,3,2181897375,My quest for early chapter books continues. I ...,Nonfiction
2,2636606,34669556,3,2178934117,Typical medievalish fantasy with dragons and t...,Fantasy
3,2636606,31193426,3,2178933041,"Nicely done, definitely diverse, lots of actio...",Graphic Novels
4,2636606,25741324,3,2177890961,"Interesting book, lots of diverse characters, ...",Childrens
5,2636606,33985145,3,2177862199,Odd format and definitely a high school title....,YA
6,2636606,27414421,3,2177861646,Library copy. Perfect for my struggling 6th gr...,Fantasy
7,2636606,25855604,3,2168911650,"Normally love this author's work, but since I ...",Childrens
8,2636606,32202503,3,2168899027,Definitely on the sad side. I agree with the a...,Childrens
9,2636606,32739712,3,2162034739,"The illustrations were charming, but there rea...",Childrens


In [76]:
## unigrams without TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
texts = list(data_text_analysis['review_text'])
# create the transform
vectorizer = CountVectorizer(decode_error='ignore',stop_words='english', max_features=1000, ngram_range=(1,1))
# tokenize and build vocab
out = vectorizer.fit_transform(texts)
i=0
data_text_analysis = data_text_analysis
for feature in vectorizer.get_feature_names():
    data_text_analysis[feature] = out.toarray()[:,i]
    i+=1

In [77]:
def nfrequent_words(x):
    return(data_text_analysis.ix[data_text_analysis['New_genre']==x].iloc[:,6:].sum().nlargest(100))
interact(nfrequent_words, x=list(np.unique(data_text_analysis.New_genre.values)))

interactive(children=(Dropdown(description=u'x', options=('Childrens', 'Christian Fiction', 'Erotic Fiction', 'Fantasy', 'Graphic Novels', 'Historical Fiction', 'Horror Suspense', 'Literary Fiction', 'Mystery Crime Detective', 'Nonfiction', 'Queer Fiction', 'Romance', 'Science Fiction', 'Thriller', 'Westerns', 'Womens Fiction', 'YA'), value='Childrens'), Output()), _dom_classes=('widget-interact',))

<function __main__.nfrequent_words>

In [65]:
## unigrams
from sklearn.feature_extraction.text import TfidfVectorizer
texts = list(data_text_analysis['review_text'])
# create the transform
vectorizer = TfidfVectorizer(decode_error='ignore',stop_words='english', max_features=1000, ngram_range=(1,1))
# tokenize and build vocab
X_unigram = vectorizer.fit_transform(texts)
i=0
data_text_analysis_unigrams = data_text_analysis
for feature in vectorizer.get_feature_names():
    data_text_analysis_unigrams[feature] = X_unigram.toarray()[:,i]
    i+=1

In [67]:
def nfrequent_words(x):
    return(data_text_analysis_unigrams.ix[data_text_analysis_unigrams['New_genre']==x].iloc[:,6:].sum().nlargest(100))
interact(nfrequent_words, x=list(np.unique(data_text_analysis.New_genre.values)))

interactive(children=(Dropdown(description=u'x', options=('Childrens', 'Christian Fiction', 'Erotic Fiction', 'Fantasy', 'Graphic Novels', 'Historical Fiction', 'Horror Suspense', 'Literary Fiction', 'Mystery Crime Detective', 'Nonfiction', 'Queer Fiction', 'Romance', 'Science Fiction', 'Thriller', 'Westerns', 'Womens Fiction', 'YA'), value='Childrens'), Output()), _dom_classes=('widget-interact',))

<function __main__.nfrequent_words>

In [70]:
## bigrams
from sklearn.feature_extraction.text import TfidfVectorizer
texts = list(data_text_analysis['review_text'])
# create the transform
vectorizer = TfidfVectorizer(decode_error='ignore',stop_words='english', max_features=1000, ngram_range=(2,2))
# tokenize and build vocab
X_bigram = vectorizer.fit_transform(texts)
i=0
data_text_analysis_bigrams = data_text_analysis
for feature in vectorizer.get_feature_names():
    data_text_analysis_bigrams[feature] = X_bigram.toarray()[:,i]
    i+=1

In [71]:
def nfrequent_words(x):
    return(data_text_analysis_bigrams.ix[data_text_analysis_bigrams['New_genre']==x].iloc[:,6:].sum().nlargest(100))
interact(nfrequent_words, x=list(np.unique(data_text_analysis.New_genre.values)))

interactive(children=(Dropdown(description=u'x', options=('Childrens', 'Christian Fiction', 'Erotic Fiction', 'Fantasy', 'Graphic Novels', 'Historical Fiction', 'Horror Suspense', 'Literary Fiction', 'Mystery Crime Detective', 'Nonfiction', 'Queer Fiction', 'Romance', 'Science Fiction', 'Thriller', 'Westerns', 'Womens Fiction', 'YA'), value='Childrens'), Output()), _dom_classes=('widget-interact',))

<function __main__.nfrequent_words>

In [73]:
## trigrams
from sklearn.feature_extraction.text import TfidfVectorizer
texts = list(data_text_analysis['review_text'])
# create the transform
vectorizer = TfidfVectorizer(decode_error='ignore',stop_words='english', max_features=1000, ngram_range=(3,3))
# tokenize and build vocab
X_trigram = vectorizer.fit_transform(texts)
i=0
data_text_analysis_trigrams = data_text_analysis
for feature in vectorizer.get_feature_names():
    data_text_analysis_trigrams[feature] = X_trigram.toarray()[:,i]
    i+=1

In [74]:
def nfrequent_words(x):
    return(data_text_analysis_trigrams.ix[data_text_analysis_trigrams['New_genre']==x].iloc[:,6:].sum().nlargest(100))
interact(nfrequent_words, x=list(np.unique(data_text_analysis.New_genre.values)))

interactive(children=(Dropdown(description=u'x', options=('Childrens', 'Christian Fiction', 'Erotic Fiction', 'Fantasy', 'Graphic Novels', 'Historical Fiction', 'Horror Suspense', 'Literary Fiction', 'Mystery Crime Detective', 'Nonfiction', 'Queer Fiction', 'Romance', 'Science Fiction', 'Thriller', 'Westerns', 'Womens Fiction', 'YA'), value='Childrens'), Output()), _dom_classes=('widget-interact',))

<function __main__.nfrequent_words>

### Distinctive words

In [16]:
## unigrams
from sklearn.feature_extraction.text import TfidfVectorizer
texts = list(data_text_analysis['review_text'])
# create the transform
vectorizer = TfidfVectorizer(decode_error='ignore',stop_words='english', max_features=1000, ngram_range=(1,1))
# tokenize and build vocab
X_unigram = vectorizer.fit_transform(texts)
i=0
data_text_analysis_unigrams = data_text_analysis
for feature in vectorizer.get_feature_names():
    data_text_analysis_unigrams[feature] = X_unigram.toarray()[:,i]
    i+=1

In [26]:
data_text_analysis[data_text_analysis['New_genre']=='Childrens'].mean()

10            0.001058
100           0.000482
12            0.000932
20            0.000284
2nd           0.001269
able          0.002473
absolutely    0.004434
account       0.000423
act           0.001131
action        0.001341
actions       0.000657
actual        0.001042
actually      0.002775
add           0.003180
added         0.001004
addition      0.003703
admit         0.000536
adorable      0.011439
adult         0.003372
adults        0.005629
adventure     0.003227
adventures    0.002280
age           0.003159
ago           0.000909
agree         0.000771
alert         0.002832
aloud         0.016077
amazing       0.004376
america       0.001244
american      0.001813
                ...   
wonderful     0.011888
wondering     0.000703
word          0.003595
words         0.010664
work          0.007833
worked        0.001418
workers       0.000065
working       0.001678
works         0.002333
world         0.006400
worth         0.001385
wouldn        0.000931
wow        

In [22]:
data_text_analysis = data_text_analysis_unigrams.drop(['user_id','book_id','user_rating','review_id','review_text'],axis=1)

In [36]:
def distinct_words(genre1):
    keyness = np.abs(data_text_analysis[data_text_analysis['New_genre']==genre1].mean()-data_text_analysis[data_text_analysis['New_genre']!=genre1].mean())/data_text_analysis.mean()  
    return(keyness.nlargest(10))
interact(distinct_words, genre1=list(np.unique(data_text_analysis.New_genre.values)))



interactive(children=(Dropdown(description=u'genre1', options=('Childrens', 'Christian Fiction', 'Erotic Fiction', 'Fantasy', 'Graphic Novels', 'Historical Fiction', 'Horror Suspense', 'Literary Fiction', 'Mystery Crime Detective', 'Nonfiction', 'Queer Fiction', 'Romance', 'Science Fiction', 'Thriller', 'Westerns', 'Womens Fiction', 'YA'), value='Childrens'), Output()), _dom_classes=('widget-interact',))

<function __main__.distinct_words>

In [41]:
keyness_Childrens = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Childrens'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Childrens'].mean())/data_text_analysis.mean()  
keyness_Christian_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Christian Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Christian Fiction'].mean())/data_text_analysis.mean()  
keyness_Erotic_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Erotic Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Erotic Fiction'].mean())/data_text_analysis.mean()  
keyness_Fantasy = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Fantasy'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Fantasy'].mean())/data_text_analysis.mean()  
keyness_Graphic_Novels = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Graphic Novels'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Graphic Novel'].mean())/data_text_analysis.mean()  
keyness_Historical_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Historical Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Historical Fiction'].mean())/data_text_analysis.mean()  
keyness_Horror_Suspense = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Horror Suspense'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Horror Suspense'].mean())/data_text_analysis.mean()  
keyness_Literary_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Literary Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Literary Fiction'].mean())/data_text_analysis.mean()  
keyness_Mystery_Crime_Detective = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Mystery Crime Detective'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Mystery Crime Detective'].mean())/data_text_analysis.mean()  
keyness_Nonfiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Nonfiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Nonfiction'].mean())/data_text_analysis.mean()  
keyness_Queer_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Queer Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Queer Fiction'].mean())/data_text_analysis.mean()  
keyness_Romance = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Romance'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Romance'].mean())/data_text_analysis.mean()  
keyness_Science_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Science Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Science Fiction'].mean())/data_text_analysis.mean()  
keyness_Thriller = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Thriller'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Thriller'].mean())/data_text_analysis.mean()  
keyness_Westerns = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Westerns'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Westerns'].mean())/data_text_analysis.mean()  
keyness_Womens_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Womens Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Womens Fiction'].mean())/data_text_analysis.mean()  
keyness_YA = np.abs(data_text_analysis[data_text_analysis['New_genre']=='YA'].mean()-data_text_analysis[data_text_analysis['New_genre']!='YA'].mean())/data_text_analysis.mean()  



KeyboardInterrupt: 

In [42]:
keyness_Science_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Science Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Science Fiction'].mean())/data_text_analysis.mean()  

In [44]:
keyness_Romance = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Romance'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Romance'].mean())/data_text_analysis.mean()  

In [45]:
keyness_Mystery_Crime_Detective = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Mystery Crime Detective'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Mystery Crime Detective'].mean())/data_text_analysis.mean()  

In [46]:
keyness_Literary_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Literary Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Literary Fiction'].mean())/data_text_analysis.mean()  

In [47]:
keyness_Horror_Suspense = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Horror Suspense'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Horror Suspense'].mean())/data_text_analysis.mean()  

In [48]:
keyness_Historical_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Historical Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Historical Fiction'].mean())/data_text_analysis.mean()  

In [49]:
keyness_Childrens = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Childrens'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Childrens'].mean())/data_text_analysis.mean()  

In [50]:
keyness_Erotic_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Erotic Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Erotic Fiction'].mean())/data_text_analysis.mean()  

In [51]:
keyness_Fantasy = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Fantasy'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Fantasy'].mean())/data_text_analysis.mean()  

In [52]:
keyness_Thriller = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Thriller'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Thriller'].mean())/data_text_analysis.mean()  

In [53]:
keyness_Womens_Fiction = np.abs(data_text_analysis[data_text_analysis['New_genre']=='Womens Fiction'].mean()-data_text_analysis[data_text_analysis['New_genre']!='Womens Fiction'].mean())/data_text_analysis.mean()  

In [54]:
keyness_YA = np.abs(data_text_analysis[data_text_analysis['New_genre']=='YA'].mean()-data_text_analysis[data_text_analysis['New_genre']!='YA'].mean())/data_text_analysis.mean()  

In [56]:
keyness_Thriller.nlargest(10)

thriller       20.370967
jack            9.686741
paced           5.437347
suspenseful     5.223114
government      4.957053
exciting        4.703335
killer          4.691147
suspense        4.386723
action          3.971498
michael         3.819169
dtype: float64